# Planet : Amazon space

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_folder = '../IPEO_Planet_project'
if not os.path.exists(data_folder):
    data_folder = input("Enter the data folder path: ")
    assert os.path.exists(data_folder), "I did not find the folder at, "+str(data_folder)

# Loading the dataset

In [5]:
labels_dt = pd.read_csv(f'{data_folder}/train_labels.csv',dtype=str)
labels_dt

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
...,...,...
40474,train_40474,clear primary
40475,train_40475,cloudy
40476,train_40476,agriculture clear primary
40477,train_40477,agriculture clear primary road


In [7]:
tags = labels_dt['tags'].str.split(expand=True).stack().unique()
tags

array(['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation',
       'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy',
       'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming',
       'selective_logging', 'blow_down'], dtype=object)

In [8]:
for tag in tags:
    a = [(tag in i.split()) for i in labels_dt['tags']]
    new_col = np.zeros(len(a),dtype=int)+a
    #print(new_col.shape)
    labels_dt[tag]=new_col
labels_dt

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,train_40474,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
40475,train_40475,cloudy,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
40476,train_40476,agriculture clear primary,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
40477,train_40477,agriculture clear primary road,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [47]:
tagg = labels_dt.tags.value_counts()
set_choice = np.zeros([len(tagg),4])
tagg

clear primary                                               13636
partly_cloudy primary                                        3630
cloudy                                                       2089
clear primary water                                          1850
agriculture clear primary road                               1680
                                                            ...  
agriculture clear cultivation cultivation road                  1
artisinal_mine bare_ground partly_cloudy primary road           1
agriculture clear conventional_mine primary road                1
clear conventional_mine habitation road                         1
agriculture partly_cloudy primary road conventional_mine        1
Name: tags, Length: 449, dtype: int64

In [50]:
for t in range(len(tagg)):
    number = tagg[t]
    set_choice[t,0] = number
    set_choice[t,1] = number//(1/0.6)
    set_choice[t,2] = (number - set_choice[t,1])//2
    set_choice[t,3] = number - set_choice[t,1] - set_choice[t,2]


In [51]:
set_choice

array([[1.3636e+04, 8.1810e+03, 2.7270e+03, 2.7280e+03],
       [3.6300e+03, 2.1770e+03, 7.2600e+02, 7.2700e+02],
       [2.0890e+03, 1.2530e+03, 4.1800e+02, 4.1800e+02],
       ...,
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00]])

In [89]:
a = np.arange(13636) # Example array.
np.random.shuffle(a)

p1 = a[:8181]
p2 = a[8181:8181+2727]
p3 = a[8181+2727:]
p2

array([11640,  4801, 12189, ...,  2251, 10497, 11165])

In [90]:
train = np.array([])
validation = np.array([])
test = np.array([])

for i in range(len(tagg)):
    a = labels_dt[labels_dt.tags==tagg.index[i]].index.array
    np.random.shuffle(a)
    p1 = a[:int(set_choice[i,1])]
    p2 = a[int(set_choice[i,1]):int(set_choice[i,1])+int(set_choice[i,2])]
    p3 = a[int(set_choice[i,1])+int(set_choice[i,2]):]
    train = np.append(train,p1)
    validation = np.append(validation,p2)
    test = np.append(test,p3)

C:\Users\Shady\anaconda3\envs\IPEO\lib\site-packages\ipykernel_launcher.py:7: UserWarning: you are shuffling a 'PandasArray' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  import sys


In [91]:
train_df = labels_dt.iloc[train]
validation_df = labels_dt.iloc[validation]
test_df = labels_dt.iloc[test]

In [92]:
repartition = np.zeros([len(tags),3])
for t in range(len(tags)):
    repartition[t,0] = (train_df[tags[t]]).sum()
    repartition[t,1] = (validation_df[tags[t]]).sum()
    repartition[t,2] = (test_df[tags[t]]).sum()

In [95]:
train_df

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
1784,train_1784,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4591,train_4591,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1513,train_1513,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
23368,train_23368,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20852,train_20852,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38272,train_38272,agriculture bare_ground clear cultivation road,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0
7967,train_7967,agriculture partly_cloudy primary road slash_burn,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0
33594,train_33594,agriculture clear conventional_mine habitation...,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0
3137,train_3137,bare_ground clear cultivation habitation prima...,0,1,0,1,1,1,1,1,0,0,0,0,1,0,0,0,0
